In [1]:
# Import libraries

import sys
import pandas as pd
import json
import geopandas as gpd
import atlite
from shapely.ops import unary_union
from shapely.geometry import Polygon

from pathlib import Path

root_path = Path(globals()['_dh'][0]).resolve().parent
sys.path.append(str(root_path))

import paths

from input.weather.weather import generate_cutout

In [2]:
# Read and set configuration

with (paths.geo_root / 'config.json').open('r') as f:
    config = json.load(f)

In [3]:
lan_code = config['weather-geo']['Västra Götalands län']['geo']
sections = config['weather-geo']['Västra Götalands län']['sections']
weather_start = config['weather-start']
weather_end = config['weather-end']

In [4]:
#Source Lantmäteriverket, data maintained by opendatasoft.com
geo_area = gpd.read_file(paths.input_root / 'geo/georef-sweden-kommun@public.geojson')

main_area = geo_area.loc[geo_area['lan_code'].isin([lan_code])]

minx, miny, maxx, maxy = main_area.total_bounds

fname = paths.weather /  f"cutout,geography={lan_code},start={weather_start},end={weather_end}.nc"

cutout = atlite.Cutout(
    path=fname,
    module="era5",
    x=slice(minx, maxx),
    y=slice(miny, maxy),
    time=slice(weather_start,weather_end),
    dx=0.125,
    dy=0.125,
    dt="3h"
)

cutout.prepare(features=['influx', 'temperature', 'wind'])

selections = {
    lan_code: gpd.GeoDataFrame(geometry=[unary_union(main_area.geometry)], crs=geo_area.crs)
}

/home/viktor/miniforge3/envs/pypsa-vgr/lib/python3.11/site-packages/atlite/cutout.py:191: UserWarning: Arguments module, x, y, time, dx, dy, dt are ignored, since cutout is already built.
  warn(


In [5]:
if sections is not None:
    for _, item in sections.items():
        if isinstance(item, list):
            kom = geo_area.loc[geo_area['kom_code'].isin(item)]
            selections[f"{'-'.join(item)}"] = gpd.GeoDataFrame(geometry=[unary_union(kom.geometry)], crs=geo_area.crs)
        else:
            kom = geo_area.loc[geo_area['kom_code'].isin([item])]
            selections[f"{item}"] = gpd.GeoDataFrame(geometry=[unary_union(kom.geometry)], crs=geo_area.crs)

{'14':                                             geometry
 0  POLYGON ((12.18585 57.56855, 12.17372 57.56457...,
 '1401':                                             geometry
 0  POLYGON ((12.22142 57.72743, 12.20827 57.72552...,
 '1402':                                             geometry
 0  POLYGON ((12.17379 57.77549, 12.11594 57.77321...,
 '1407':                                             geometry
 0  POLYGON ((11.73133 57.77120, 11.68180 57.78679...,
 '1415':                                             geometry
 0  POLYGON ((11.98361 58.14914, 11.97512 58.14896...,
 '1419':                                             geometry
 0  POLYGON ((11.73680 57.97853, 11.75423 57.98836...,
 '1421':                                             geometry
 0  POLYGON ((10.89178 58.17594, 10.95957 58.04822...,
 '1427':                                             geometry
 0  POLYGON ((11.46761 58.47046, 11.46095 58.47201...,
 '1430':                                             geometry
 0  

In [ ]:


# EEZ (Economical zone)
shapefile_path = paths.input_root / 'geo/Ekonomiska_zonens_yttre_avgränsningslinjer/Ekonomiska_zonens_yttre_avgränsningslinjer_linje.shp'
eez_shape = gpd.read_file(shapefile_path).to_crs(geo_area.crs)
min_x, min_y, max_x, max_y = eez_shape.total_bounds
# Arbitrarily using min/max from cutout or eez to visualize it on VGR region
bounding_box = Polygon([(min_x, miny), (min_x, maxy), (maxx, maxy), (maxx, miny)])
bounds = gpd.GeoDataFrame(geometry=[bounding_box], crs=geo_area.crs) 
eez = gpd.overlay(eez_shape, bounds, how='intersection')
eez.to_crs(geo_area.crs)

index = pd.to_datetime(cutout.coords['time'])